In [1]:
from config.simpleconv_config import SimpleConvConfig
from models.simpleconv import SimpleConv
from studies.study_factory import StudyFactory
from utils.pre_processor import PreProcessor
from utils.fetch import fetch_audio_and_brain_pairs

import torch

In [2]:
# model = SimpleConv(SimpleConvConfig(transformer_layers=0))

In [3]:
brain_sample_rate = 100

study = StudyFactory().create_study("gwilliams", path="data/gwilliams")
pre_processor = PreProcessor(
    brain_sample_rate=brain_sample_rate,
)
frequency_bands = {"all": (0.5, 100)}
subject, task, session = 0, 0, 0
seed = 42
max_random_shift = 1
window_size = 4
n_jobs = -1

In [4]:
# Get attributes for the recording
raw, word_events, sound_events = study.clean_recording(
    subject, task, session, n_jobs=n_jobs
)

In [5]:
import mne
mne.find_layout(raw.info).pos.shape

(208, 4)

In [6]:
raw.get_data().shape

(208, 396000)

In [10]:
brain_segments, audio_segments, layout = fetch_audio_and_brain_pairs(
    subject=subject,
    task=task,
    session=session,
    max_random_shift=max_random_shift,
    window_size=window_size,
    study=study,
    pre_processor=pre_processor,
    frequency_bands=frequency_bands,
    audio_sample_rate=16000,
    hop_length=160,
    n_jobs=n_jobs,
)

In [11]:
brain_segments['all'].shape, audio_segments.shape, layout.shape

(torch.Size([197, 208, 400]),
 torch.Size([197, 128, 400]),
 torch.Size([208, 2]))